Hey, un pequeño tutorial en como conectarse

2024-11-19 20:06:18 [I 2024-11-20 02:06:18.604 LabApp] Extension Manager is 'pypi'.
2024-11-19 20:06:18 [I 2024-11-20 02:06:18.606 ServerApp] jupyterlab | extension was successfully loaded.
2024-11-19 20:06:18 [I 2024-11-20 02:06:18.609 ServerApp] jupyterlab_git | extension was successfully loaded.
2024-11-19 20:06:18 [I 2024-11-20 02:06:18.613 ServerApp] nbclassic | extension was successfully loaded.
2024-11-19 20:06:18 [I 2024-11-20 02:06:18.670 ServerApp] nbdime | extension was successfully loaded.
2024-11-19 20:06:18 [I 2024-11-20 02:06:18.672 ServerApp] notebook | extension was successfully loaded.
2024-11-19 20:06:18 [I 2024-11-20 02:06:18.672 ServerApp] Serving notebooks from local directory: /home/jovyan
2024-11-19 20:06:18 [I 2024-11-20 02:06:18.672 ServerApp] Jupyter Server 2.8.0 is running at:
2024-11-19 20:06:18 [I 2024-11-20 02:06:18.673 ServerApp] http://3b233796100e:8888/lab?token=6c2c103b09de5ede89ba7efeafc354b1d1745cb9ec496ac5
2024-11-19 20:06:18 [I 2024-11-20 02:06:18.673 ServerApp]     http://127.0.0.1:8888/lab?token=6c2c103b09de5ede89ba7efeafc354b1d1745cb9ec496ac5
2024-11-19 20:06:18 [I 2024-11-20 02:06:18.673 ServerApp] Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
2024-11-19 20:06:18 [C 2024-11-20 02:06:18.675 ServerApp] 
2024-11-19 20:06:18     
2024-11-19 20:06:18     To access the server, open this file in a browser:
2024-11-19 20:06:18         file:///home/jovyan/.local/share/jupyter/runtime/jpserver-7-open.html
2024-11-19 20:06:18     Or copy and paste one of these URLs:
2024-11-19 20:06:18         http://3b233796100e:8888/lab?token=6c2c103b09de5ede89ba7efeafc354b1d1745cb9ec496ac5
2024-11-19 20:06:18         http://127.0.0.1:8888/lab?token=6c2c103b09de5ede89ba7efeafc354b1d1745cb9ec496ac5
2024-11-19 20:06:20 [I 2024-11-20 02:06:20.006 ServerApp] Skipped non-installed server(s): bash-language-server, dockerfile-language-server-nodejs, javascript-typescript-langserver, jedi-language-server, julia-language-server, pyright, python-language-server, python-lsp-server, r-languageserver, sql-language-server, texlab, typescript-language-server, unified-language-server, vscode-css-languageserver-bin, vscode-html-languageserver-bin, vscode-json-languageserver-bin, yaml-language-server

Se ve feo pero de esto es como se conecta al server

http://127.0.0.1:8888/lab?token=6c2c103b09de5ede89ba7efeafc354b1d1745cb9ec496ac5

http://127.0.0.1:8888/lab

en este caso este es el token que genero, con esto es que uno conecta el vscode al jupyter solo por si acaso


In [2]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SparkSQL").master('local').getOrCreate()

In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = (
    '--jars /usr/local/spark/jars/neo4j-connector-apache-spark_2.12-5.2.0_for_spark_3.jar,'
    '/usr/local/spark/jars/postgresql-42.7.4.jar pyspark-shell'
)

In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

: 

In [3]:
data1 = [(1, 'a', 111), (2, 'b', 222), (3, 'c', 333)]
columns = ['id', 'name', 'age']
cdf = spark.createDataFrame(data1, columns)
cdf.show()

+---+----+---+
| id|name|age|
+---+----+---+
|  1|   a|111|
|  2|   b|222|
|  3|   c|333|
+---+----+---+



ds = (
    spark.read.format("org.neo4j.spark.DataSource")
    .option("url", url)
    .option("authentication.basic.username", username)
    .option("authentication.basic.password", password)
    .option("labels", "Product")
    .load()
)

ds.show(n=1000, truncate=False)

In [ ]:
from pyspark.sql import SparkSession

# Replace with the actual connection URI and credentials
url = "bolt://neo4j:7687"
username = "neo4j"
password = "password"
dbname = "neo4j"

spark = (
    SparkSession.builder.appName("Neo4jIntegration")
    .config("spark.jars", "/home/jovyan/neo4j-connector-apache-spark_2.12-5.2.0_for_spark_3.jar")
    .config("spark.neo4j.bolt.url", url)
    .config("spark.neo4j.bolt.user", username)
    .config("spark.neo4j.bolt.password", password)
    .config("spark.neo4j.database", dbname)
    .getOrCreate()
)


Consulta1 = (
    spark.read.format("org.neo4j.spark.DataSource")
    .option("url", url)
    .option("authentication.basic.username", username)
    .option("authentication.basic.password", password)
    .option("query", """
        MATCH (u:User)
        OPTIONAL MATCH (u)-[r:PURCHASED]->()
        RETURN id(u) AS id, u.name AS name, u.gender AS gender, 
               u.address AS address, u.job_title AS job, 
               toFloat(sum(toFloat(substring(r.transaction_amount, 1)))) AS Total_spent
        ORDER BY Total_spent DESC
    """)
    .load()
)


Consulta2 = (
    spark.read.format("org.neo4j.spark.DataSource")
    .option("url", url)
    .option("authentication.basic.username", username)
    .option("authentication.basic.password", password)
    .option("query", """
        MATCH (p:Product)
        OPTIONAL MATCH (p)<-[r:PURCHASED]-()
        RETURN id(p) AS id, labels(p) AS labels, p.product_size AS product_size, p.product_line AS product_line, 
               p.product_id AS product_id, p.product_class AS product_class, p.product_brand AS product_brand, 
               count(r) AS purchase_count
        ORDER BY purchase_count DESC
    """)
    .load()
)

Consulta3 = (
    spark.read.format("org.neo4j.spark.DataSource")
    .option("url", url)
    .option("authentication.basic.username", username)
    .option("authentication.basic.password", password)
    .option("query", """
        MATCH (u:User)
        OPTIONAL MATCH (u)-[r:PURCHASED]->()
        RETURN id(u) AS id, u.name AS name, u.gender AS gender, 
               u.address AS address, u.job_title AS job, 
               toFloat(sum(toFloat(substring(r.transaction_amount, 1)))) AS Total_spent,
               count(r) AS transaction_count,
               CASE WHEN count(r) > 0 THEN toFloat(sum(toFloat(substring(r.transaction_amount, 1))) / count(r)) ELSE toFloat(0) END AS Average_spent
        ORDER BY Average_spent DESC
    """)
    .load()
)



Consulta4 = (
    spark.read.format("org.neo4j.spark.DataSource")
    .option("url", url)
    .option("authentication.basic.username", username)
    .option("authentication.basic.password", password)
    .option("query", """
        MATCH (u:User)
        OPTIONAL MATCH (u)-[r:PURCHASED]->()
        RETURN id(u) AS id, u.name AS name, u.gender AS gender, 
               u.address AS address, u.job_title AS job, 
               count(r) AS purchase_count
        ORDER BY purchase_count DESC
    """)
    .load()
)

Consulta4.show(n=1000, truncate=False)


+----+------------------------+------+-----------------------------+------------------------------------+--------------+
|id  |name                    |gender|address                      |job                                 |purchase_count|
+----+------------------------+------+-----------------------------+------------------------------------+--------------+
|295 |Hal Braddon             |Male  |567 Scott Park               |Administrative Officer              |14            |
|3525|Frazer Searston         |Male  |4297 Emmet Lane              |NA                                  |14            |
|1333|Jillie Fyndon           |Female|8142 Tomscot Drive           |Programmer Analyst IV               |14            |
|3486|Rose Milsom             |Female|78258 Hanover Plaza          |Payment Adjustment Coordinator      |13            |
|1975|Tobit                   |Male  |7916 Clyde Gallagher Place   |NA                                  |13            |
|508 |Margie Tillyer          |F

In [6]:
postgres_url = "jdbc:postgresql://dbpostgres/summaries"
postgres_properties = {
    "user": "user",
    "password": "password",
    "driver": "org.postgresql.Driver"
}

Consulta1.write \
    .format("jdbc") \
    .option("url", postgres_url) \
    .option("dbtable", "total_spent_per_customer") \
    .options(**postgres_properties) \
    .mode("overwrite") \
    .save()

Consulta2.write \
    .format("jdbc") \
    .option("url", postgres_url) \
    .option("dbtable", "product_purchase_count") \
    .options(**postgres_properties) \
    .mode("overwrite") \
    .save()

Consulta3.write \
    .format("jdbc") \
    .option("url", postgres_url) \
    .option("dbtable", "average_spend_per_customer") \
    .options(**postgres_properties) \
    .mode("overwrite") \
    .save()

Consulta4.write \
    .format("jdbc") \
    .option("url", postgres_url) \
    .option("dbtable", "transaction_count_per_customer") \
    .options(**postgres_properties) \
    .mode("overwrite") \
    .save()
